In [46]:
import sqlite3
import pandas as pd

# 메모리에 임시 DB 생성
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# 1. 주문 테이블 생성 (날짜 컬럼 추가)
cur.execute('''
CREATE TABLE orders (
    order_id VARCHAR(10),
    user_id VARCHAR(10),
    order_date DATE,
    amount INTEGER
)
''')

# 2. 실전 같은 더미 데이터 (2023년~2024년 데이터)
data = [
    ('O001', 'User_A', '2024-01-01', 50000),
    ('O002', 'User_A', '2024-01-03', 30000),
    ('O003', 'User_B', '2023-05-01', 100000),
    ('O004', 'User_B', '2023-06-15', 20000),
    ('O005', 'User_C', '2024-01-02', 5000),
    ('O006', 'User_C', '2024-01-03', 10000),
    ('O007', 'User_D', '2023-12-10', 150000),
    ('O008', 'User_A', '2024-01-05', 70000),
]

cur.executemany('INSERT INTO orders VALUES (?, ?, ?, ?)', data)
conn.commit()

print("✅ Day 6 프로젝트 데이터 준비 완료!")

✅ Day 6 프로젝트 데이터 준비 완료!


In [47]:
sql_rfm = """
WITH RFM_BASE AS (
    SELECT
        user_id, Monetary,
        CASE
            WHEN Recency <= 10 THEN 5
            WHEN Recency <= 30 THEN 3
            ELSE 1
        END AS R_Score,
        CASE
            WHEN Frequency >= 3 THEN 5
            WHEN Frequency >= 2 THEN 3
            ELSE 1
        END AS F_Score,
        CASE
            WHEN Monetary >= 100000 THEN 5
            WHEN Monetary >= 30000 THEN 3
            ELSE 1
        END AS M_Score
    FROM (
        SELECT
            user_id, 
            CAST(julianday('2024-01-06') - julianday(MAX(order_date)) AS INTEGER) AS Recency,
            COUNT(user_id) AS Frequency, 
            SUM(amount) AS Monetary
        FROM orders
        GROUP BY user_id
    )
),

RFM_FINAL AS (
    SELECT
        user_id, Monetary,
        CASE
            WHEN R_Score = 5 AND (F_Score >= 3 OR M_Score >= 3) THEN 'VIP'
            WHEN R_Score = 1 AND (F_Score >= 3 OR M_Score >= 3) THEN '이탈 위험'
            WHEN R_Score >= 3 AND F_Score = 1 THEN '신규'
            ELSE '일반'
        END AS Class
    FROM RFM_BASE
)

SELECT
    Class,
    COUNT(user_id) AS User_Count,
    SUM(Monetary) AS Total_Revenue,
    ROUND(AVG(Monetary), 0) AS Avg_Revenue
FROM RFM_FINAL
GROUP BY Class
ORDER BY Total_Revenue DESC;
"""
print(pd.read_sql(sql_rfm, conn))

   Class  User_Count  Total_Revenue  Avg_Revenue
0    VIP           2         165000      82500.0
1     신규           1         150000     150000.0
2  이탈 위험           1         120000     120000.0


In [48]:
# users 테이블 생성 (JOIN 연습용)
cur.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id VARCHAR(10) PRIMARY KEY,
    name VARCHAR(20),
    age INTEGER,
    city VARCHAR(20)
)
''')

# 데이터 넣기 (User_A, B, C는 있는데 D는 없는 상황)
# User_D는 '비회원 주문' 케이스로 남겨둡니다.
user_data = [
    ('User_A', '철수', 30, '서울'),
    ('User_B', '영희', 25, '부산'),
    ('User_C', '민수', 40, '서울'),
    ('User_E', '지수', 22, '대구') 
]

cur.executemany('INSERT OR IGNORE INTO users VALUES (?, ?, ?, ?)', user_data)
conn.commit()

print("✅ 테스트 준비 완료! users 테이블이 생성되었습니다.")

✅ 테스트 준비 완료! users 테이블이 생성되었습니다.


In [49]:
sql_q1 = """
SELECT users.name, SUM(orders.amount) AS total_spent
FROM users
INNER JOIN orders
ON users.user_id = orders.user_id
WHERE users.city = '서울'
GROUP BY users.name
"""
print(pd.read_sql(sql_q1, conn))

  name  total_spent
0   민수        15000
1   철수       150000


In [50]:
sql_q2 = """
SELECT
    CASE
        WHEN amount >= 50000 THEN 'HIGH'
        ELSE 'LOW'
    END AS Class,
    COUNT(orders.order_id) AS Order_Count
FROM orders
GROUP BY Class
"""
print(pd.read_sql(sql_q2, conn))

  Class  Order_Count
0  HIGH            4
1   LOW            4


In [51]:
sql_q3 = """
WITH User_Sales AS (
    SELECT user_id, SUM(amount) AS Total
    FROM orders
    GROUP BY user_id
)
SELECT users.name, User_Sales.Total
FROM User_Sales
INNER JOIN users
ON User_Sales.user_id = users.user_id
WHERE User_Sales.Total >= 100000
"""
print(pd.read_sql(sql_q3, conn))

  name   Total
0   철수  150000
1   영희  120000


In [52]:
sql_window1 = """
SELECT
    user_id,
    order_date,
    amount,
    RANK() OVER (
        PARTITION BY user_id
        ORDER BY amount DESC
    ) AS ranking
FROM orders
ORDER BY user_id, ranking;
"""
print(pd.read_sql(sql_window1, conn))

  user_id  order_date  amount  ranking
0  User_A  2024-01-05   70000        1
1  User_A  2024-01-01   50000        2
2  User_A  2024-01-03   30000        3
3  User_B  2023-05-01  100000        1
4  User_B  2023-06-15   20000        2
5  User_C  2024-01-03   10000        1
6  User_C  2024-01-02    5000        2
7  User_D  2023-12-10  150000        1


In [53]:
sql_window2 = """
SELECT
    user_id,
    order_date,
    amount AS current_amount,
    LAG(amount) OVER (
        PARTITION BY user_id
        ORDER BY order_date
    ) AS prev_amount
FROM orders
"""
print(pd.read_sql(sql_window2, conn))

  user_id  order_date  current_amount  prev_amount
0  User_A  2024-01-01           50000          NaN
1  User_A  2024-01-03           30000      50000.0
2  User_A  2024-01-05           70000      30000.0
3  User_B  2023-05-01          100000          NaN
4  User_B  2023-06-15           20000     100000.0
5  User_C  2024-01-02            5000          NaN
6  User_C  2024-01-03           10000       5000.0
7  User_D  2023-12-10          150000          NaN


In [54]:
sql_cycle = """
SELECT
    user_id,
    order_date,
    LAG(order_date) OVER (PARTITION BY user_id ORDER BY order_date) AS prev_date,
    CAST(
        julianday(order_date) - julianday(LAG(order_date) OVER (PARTITION BY user_id ORDER BY order_date)) AS INTEGER
    ) AS days_since_last
FROM orders
ORDER BY user_id, order_date;
"""
print(pd.read_sql(sql_cycle, conn))

  user_id  order_date   prev_date  days_since_last
0  User_A  2024-01-01        None              NaN
1  User_A  2024-01-03  2024-01-01              2.0
2  User_A  2024-01-05  2024-01-03              2.0
3  User_B  2023-05-01        None              NaN
4  User_B  2023-06-15  2023-05-01             45.0
5  User_C  2024-01-02        None              NaN
6  User_C  2024-01-03  2024-01-02              1.0
7  User_D  2023-12-10        None              NaN


In [55]:
sql_running = """
SELECT
    user_id,
    order_date,
    amount,
    -- 누적 합계 구하기 (Running Total)
    SUM(amount) OVER (
        PARTITION BY user_id      
        ORDER BY order_date       -- 날짜 순서대로 차곡차곡 더해라
    ) AS cumulative_sales
FROM orders
ORDER BY user_id, order_date;
"""
print(pd.read_sql(sql_running, conn))

  user_id  order_date  amount  cumulative_sales
0  User_A  2024-01-01   50000             50000
1  User_A  2024-01-03   30000             80000
2  User_A  2024-01-05   70000            150000
3  User_B  2023-05-01  100000            100000
4  User_B  2023-06-15   20000            120000
5  User_C  2024-01-02    5000              5000
6  User_C  2024-01-03   10000             15000
7  User_D  2023-12-10  150000            150000


In [56]:
sql_moving_avg = """
SELECT
    user_id,
    order_date,
    amount,
    AVG(amount) OVER (
        PARTITION BY user_id
        ORDER BY order_date
        ROWS BETWEEN 1 PRECEDING AND CURRENT ROW
    ) AS moving_avg
FROM orders
ORDER BY user_id, order_date;
"""
print(pd.read_sql(sql_moving_avg, conn))

  user_id  order_date  amount  moving_avg
0  User_A  2024-01-01   50000     50000.0
1  User_A  2024-01-03   30000     40000.0
2  User_A  2024-01-05   70000     50000.0
3  User_B  2023-05-01  100000    100000.0
4  User_B  2023-06-15   20000     60000.0
5  User_C  2024-01-02    5000      5000.0
6  User_C  2024-01-03   10000      7500.0
7  User_D  2023-12-10  150000    150000.0


In [59]:
sql_vip_date = """
WITH Running_Stats AS (
    SELECT
        user_id,
        order_date,
        SUM(amount) OVER (
            PARTITION BY user_id
            ORDER BY order_date
        ) AS cum_sales  
    FROM orders
)
SELECT
    user_id,
    MIN(order_date) AS vip_achieved_date
FROM Running_Stats
WHERE cum_sales >= 100000
GROUP BY user_id
"""
print(pd.read_sql(sql_vip_date, conn))

  user_id vip_achieved_date
0  User_A        2024-01-05
1  User_B        2023-05-01
2  User_D        2023-12-10
